# FastPitch SpeakerAdaptation

This notebook is designed to provide a guide on how to run FastPitch Speaker Adaptation Pipeline. It contains the following sections:
1. **Pre-train FastPitch on multi-speaker data**: pre-train a multi-speaker FastPitch
* Dataset Preparation: download dataset and extract manifest files.
* Preprocessing: add absolute audio paths and normalized texts in manifest, calculate pitch stats, extract speaker embedding from pre-trained speaker-verification (SV) model.
* Training: pre-train multispeaker FastPitch with additional four components including 1) Looked-up speaker embedding 2) Pre-trained SV speaker embedding 3) Global Style Tokens 4) Conditional Layer Normalization.
* Transform pretrained checkpoint to adapter-compatible model: transform original checkpoint configs to adapter-compatible configs.
2. **Fine-tune HiFiGAN on multi-speaker data**: fine-tune a vocoder for the pre-trained multi-speaker FastPitch
* Dataset Preparation: extract mel-spectrograms from pre-trained FastPitch.
* Training: fine-tune HiFiGAN with pre-trained multi-speaker data.
3. **Fine-tune FastPitch on adaptation data**: fine-tune pre-trained multi-speaker FastPitch for a new speaker
* Dataset Preparation: download dataset and extract manifest files. (duration more than 15 mins)
* Preprocessing: add absolute audio paths and normalized texts in manifest, calculate pitch stats, extract speaker embedding from pre-trained speaker-verification (SV) model.
* Training: fine-tune frozen multispeaker FastPitch with trainable adapters and weighted looked-up speaker embedding.
4. **Fine-tune HiFiGAN on adaptation data**: fine-tune a vocoder for the fine-tuned multi-speaker FastPitch
* Dataset Preparation: extract mel-spectrograms from fine-tuned FastPitch.
* Training: fine-tune HiFiGAN with fine-tuned adaptation data.
5. **Inference**: generate speech from adpated FastPitch
* Load Model: load pre-trained multi-speaker FastPitch with fine-tuned adapters and weighted looked-up speaker embedding.
* Output Audio: generate audio files.

# License

> Copyright 2023 NVIDIA. All Rights Reserved.
> 
> Licensed under the Apache License, Version 2.0 (the "License");
> you may not use this file except in compliance with the License.
> You may obtain a copy of the License at
> 
>     http://www.apache.org/licenses/LICENSE-2.0
> 
> Unless required by applicable law or agreed to in writing, software
> distributed under the License is distributed on an "AS IS" BASIS,
> WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
> See the License for the specific language governing permissions and
> limitations under the License.

In [ ]:
"""
You can either run this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.
Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies# .
"""
BRANCH = 'main'
# # If you're using Colab and not running locally, uncomment and run this cell.
# !apt-get install sox libsndfile1 ffmpeg
# !pip install wget unidecode pynini==2.1.4 scipy==1.7.3
# !python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

In [ ]:
# Store all python script
codedir = 'NeMoTTS' 
# Store all manifest and audios
datadir = 'NeMoTTS_dataset'
# Store all related text-normalized files
normdir = 'NeMoTTS_normalize_files'
# Store all supplementary files
suppdir = "NeMoTTS_sup_data"
# Store all config files
confdir = "NeMoTTS_conf"
# Store all training logs
logsdir = "NeMoTTS_logs"
# Store all mel-spectrograms for vocoder training
melsdir = "NeMoTTS_mels"
# Store all generated audios
gensdir = "NeMoTTS_gens"

In [ ]:
!wandb login #paste_wandb_apikey_here

In [ ]:
import os
import json
import nemo
import torch
import numpy as np

from pathlib import Path
from tqdm import tqdm

# 1. Pre-train FastPitch on multi-speaker data

## a. Dataset Preparation
For our tutorial, we use Hi-Fi Multi-Speaker English TTS (Hi-Fi TTS) dataset with 10 speakers. The audios have 44100 kHz sampling rate. You can read more about dataset [here](https://arxiv.org/abs/2104.01497).

In [ ]:
!mkdir {codedir} && cd {codedir} && wget https://raw.githubusercontent.com/nvidia/NeMo/$BRANCH/scripts/dataset_processing/tts/hifitts/get_data.py

In [ ]:
!(mkdir {datadir} && \
  cd {codedir} && \
  python get_data.py \
        --data-root ../{datadir}/ )

In [ ]:
manidir = f"{datadir}/hi_fi_tts_v0"
!ls {manidir}

For simplicity, we use original dev set as training set and original test set as validation set.

In [ ]:
train_manifest = os.path.abspath(os.path.join(manidir, 'dev_manifest.json'))
valid_manifest = os.path.abspath(os.path.join(manidir, 'test_manifest.json'))

## b. Preprocessing

In [ ]:
# additional files
!mkdir -p {normdir} && cd {normdir} \
&& wget https://raw.githubusercontent.com/nvidia/NeMo/$BRANCH/scripts/tts_dataset_files/cmudict-0.7b_nv22.10 \
&& wget https://raw.githubusercontent.com/nvidia/NeMo/$BRANCH/scripts/tts_dataset_files/heteronyms-052722 \
&& wget 

### Add absoluate audio path in manifest

In [ ]:
def json_reader(filename):
    lines = []
    with open(filename) as f:
        for line in f: lines.append(json.loads(line))
    return lines

def json_writer(manifest, filename):
    with open(filename, 'w') as fout:
        for m in manifest: fout.write(json.dumps(m) + '\n') 

In [ ]:
train_datas = json_reader(train_manifest)
for m in train_datas: m['audio_filepath'] = os.path.abspath(os.path.join(manidir, m['audio_filepath']))
json_writer(train_datas, train_manifest)

valid_datas = json_reader(valid_manifest)
for m in valid_datas: m['audio_filepath'] = os.path.abspath(os.path.join(manidir, m['audio_filepath']))
json_writer(valid_datas, valid_manifest)

### Add normalized text in manifest

In [ ]:
cd {codedir} && wget https://raw.githubusercontent.com/nvidia/NeMo/$BRANCH/scripts/dataset_processing/add_normalized_text.py

In [ ]:
!(cd {codedir} && python add_normalized_text.py --src {train_manifest} --dst {train_manifest}) 

In [ ]:
!(cd {codedir} && python add_normalized_text.py --src {valid_manifest} --dst {valid_manifest}) 

### Calculate Pitch Stats

In [ ]:
import librosa
from pathlib import Path
from nemo.collections.asr.parts.preprocessing.features import WaveformFeaturizer
from nemo.collections.tts.parts.utils.tts_dataset_utils import get_base_dir

In [ ]:
def get_pitch(sample):    
    rel_audio_path = Path(sample["audio_filepath"]).relative_to(base_data_dir).with_suffix("")
    rel_audio_path_as_text_id = str(rel_audio_path).replace("/", "_")
    pitch_filepath = os.path.join(pitch_dir, f"{rel_audio_path_as_text_id}.pt")
    
    if os.path.exists(pitch_filepath):
        pitch = torch.load(pitch_filepath).numpy()

    else:
        features = wave_model.process(
            sample["audio_filepath"]
        )
        voiced_tuple = librosa.pyin(
            features.numpy(),
            fmin=librosa.note_to_hz('C2'),
            fmax=librosa.note_to_hz('C7'),
            frame_length=2048,
            sr=44100,
            fill_na=0.0,
        )
        pitch = voiced_tuple[0]
        torch.save(torch.from_numpy(pitch).float(), pitch_filepath)
    
    return pitch

In [ ]:
wave_model = WaveformFeaturizer(sample_rate=44100)
pitch_dir = os.path.join(suppdir, 'pitch')
os.makedirs(pitch_dir, exist_ok=True)

train_pitchs = []
train_datas = json_reader(train_manifest)
base_data_dir = get_base_dir([item["audio_filepath"] for item in train_datas])
for m in tqdm(train_datas): train_pitchs.append(get_pitch(m))
    
valid_datas = json_reader(valid_manifest)
base_data_dir = get_base_dir([item["audio_filepath"] for item in valid_datas])
for m in tqdm(valid_datas): get_pitch(m)

train_pitchs = np.concatenate(train_pitchs)
pitch_mean = float(np.mean(train_pitchs))
pitch_std = float(np.std(train_pitchs))

with open(os.path.join(manidir, 'pitch_stats.json'), 'w') as f:
    json.dump({'pitch':[pitch_mean, pitch_std]}, f)

### Extract speaker embedding from pre-trained speaker-verification model 

In [ ]:
import nemo.collections.asr as nemo_asr

In [ ]:
verification_model = nemo_asr.models.EncDecSpeakerLabelModel.from_pretrained('titanet_large')
verification_model.eval().cuda()

In [ ]:
train_datas = json_reader(train_manifest)
speaker2id = {s: _id for _id, s in enumerate(set([m['speaker'] for m in train_datas]))}

for m in train_datas: 
    old_id, new_id = m['speaker'], speaker2id[m['speaker']]
    m['old_speaker'] = old_id
    m['speaker'] = new_id
    m['label'] = new_id
    
json_writer(train_datas, train_manifest)

valid_datas = json_reader(valid_manifest)
for m in valid_datas: 
    old_id, new_id = m['speaker'], speaker2id[m['speaker']]
    m['old_speaker'] = old_id
    m['speaker'] = new_id
    m['label'] = new_id
    
json_writer(valid_datas, valid_manifest)

In [ ]:
embedding_path = os.path.join(suppdir, 'speaker_embedding')
os.makedirs(embedding_path, exist_ok=True)

# Train
embs, *_ = verification_model.batch_inference(manifest_filepath=train_manifest,
                                              batch_size=64, 
                                              sample_rate=16000, 
                                              device='cuda')
np.save(os.path.join(embedding_path, 'train.npy'), embs)

# Valid
embs, *_ = verification_model.batch_inference(manifest_filepath=valid_manifest,
                                              batch_size=64, 
                                              sample_rate=16000, 
                                              device='cuda')
np.save(os.path.join(embedding_path, 'valid.npy'), embs)

## c. Training

In [ ]:
!mkdir -p {confdir} && cd {confdir} \
&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/tts/conf/fastpitch_speaker_adaptation.yaml

In [ ]:
!cd {codedir} \
&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/tts/fastpitch.py

In [ ]:
# Normally 200 epochs

!(python {codedir}/fastpitch.py \
  --config-path={os.path.abspath(confdir)} \
  --config-name=fastpitch_speaker_adaptation.yaml \
  sample_rate=44100 \
  train_dataset={train_manifest} \
  validation_datasets={valid_manifest} \
  sup_data_types="['align_prior_matrix', 'pitch', 'speaker_id','gst_ref_audio','speaker_embedding']" \
  sup_data_path={suppdir} \
  +init_from_pretrained_model="tts_en_fastpitch" \
  pitch_mean={pitch_mean} \
  pitch_std={pitch_std} \
  phoneme_dict_path={normdir}/cmudict-0.7b_nv22.10 \
  heteronyms_path={normdir}/heteronyms-052722 \
  model.n_speakers=10 \
  model.speaker_emb_condition_prosody=True \
  model.speaker_emb_condition_decoder=True \
  model.speaker_emb_condition_aligner=True \
  model.speaker_emb_condition_layernm=True \
  model.speaker_embedding_dim=192 \
  model.train_ds.dataloader_params.batch_size=8 \
  model.validation_ds.dataloader_params.batch_size=8 \
  model.train_ds.dataloader_params.num_workers=8 \
  model.validation_ds.dataloader_params.num_workers=8 \
  model.train_ds.dataset.max_duration=20 \
  model.validation_ds.dataset.max_duration=20 \
  model.validation_ds.dataset.min_duration=0.1 \
  +model.text_tokenizer.add_blank_at=True \
  +model.train_ds.dataset.speaker_embedding_path={suppdir}/speaker_embedding/train.npy \
  +model.validation_ds.dataset.speaker_embedding_path={suppdir}/speaker_embedding/valid.npy \
  exp_manager.exp_dir={logsdir} \
  +exp_manager.create_wandb_logger=True \
  +exp_manager.wandb_logger_kwargs.name="tutorial-FastPitch-pretrain-multispeaker" \
  +exp_manager.wandb_logger_kwargs.project="NeMo" \
  trainer.check_val_every_n_epoch=1 \
  trainer.log_every_n_steps=1 \
  trainer.max_epochs=5 \
  trainer.devices=-1 \
  trainer.strategy=ddp \
  trainer.precision=32 \
)

## d. Transform pretrained checkpoint to adapter-compatible model

In [ ]:
from nemo.core import adapter_mixins
from nemo.collections.tts.modules.submodules import WeightedSpeakerEmbedding
from omegaconf import DictConfig, OmegaConf, open_dict

In [ ]:
def update_model_config_to_support_adapter(config) -> DictConfig:
    with open_dict(config):
        enc_adapter_metadata = adapter_mixins.get_registered_adapter(config.input_fft._target_)
        if enc_adapter_metadata is not None:
            config.input_fft._target_ = enc_adapter_metadata.adapter_class_path

        dec_adapter_metadata = adapter_mixins.get_registered_adapter(config.output_fft._target_)
        if dec_adapter_metadata is not None:
            config.output_fft._target_ = dec_adapter_metadata.adapter_class_path

        pitch_predictor_adapter_metadata = adapter_mixins.get_registered_adapter(config.pitch_predictor._target_)
        if pitch_predictor_adapter_metadata is not None:
            config.pitch_predictor._target_ = pitch_predictor_adapter_metadata.adapter_class_path

        duration_predictor_adapter_metadata = adapter_mixins.get_registered_adapter(config.duration_predictor._target_)
        if duration_predictor_adapter_metadata is not None:
            config.duration_predictor._target_ = duration_predictor_adapter_metadata.adapter_class_path

        aligner_adapter_metadata = adapter_mixins.get_registered_adapter(config.alignment_module._target_)
        if aligner_adapter_metadata is not None:
            config.alignment_module._target_ = aligner_adapter_metadata.adapter_class_path
    return config

In [ ]:
ckpt = "YOUR_PRETRAINED_FASTPITCH_CHECKPOINT"
state = torch.load(ckpt)
state['hyper_parameters']['cfg'] = update_model_config_to_support_adapter(state['hyper_parameters']['cfg'])
torch.save(state, ckpt)

# 2. Fine-tune HiFiGAN on multi-speaker data

## a. Dataset Preparation

In [ ]:
from nemo.collections.tts.parts.utils.tts_dataset_utils import BetaBinomialInterpolator
from nemo.collections.tts.models import FastPitchModel
from collections import defaultdict
import random

In [ ]:
def gen_spectrogram(index, manifest, speaker_embedding, speaker_to_index, mel_dir, pitch_dir, base_data_dir):
    
    record = manifest[index]
    audio_file = record["audio_filepath"]
    
    if '.wav' in audio_file:
        save_path = os.path.abspath(os.path.join(mel_dir, audio_file.split("/")[-1].replace(".wav", ".npy")))
    
    if '.flac' in audio_file:
        save_path = os.path.abspath(os.path.join(mel_dir, audio_file.split("/")[-1].replace(".flac", ".npy")))
    
    if os.path.exists(save_path):
        return save_path
    
    if "normalized_text" in record:
        text = spec_model.parse(record["normalized_text"], normalize=False)
    else:
        text = spec_model.parse(record['text'])
        
    text_len = torch.tensor(text.shape[-1], dtype=torch.long, device=spec_model.device).unsqueeze(0)
    
    audio = wave_model.process(audio_file).unsqueeze(0).to(device=spec_model.device)
    audio_len = torch.tensor(audio.shape[1]).long().unsqueeze(0).to(device=spec_model.device)
    spect, spect_len = spec_model.preprocessor(input_signal=audio, length=audio_len) 
    
    rel_audio_path = Path(audio_file).relative_to(base_data_dir).with_suffix("")
    rel_audio_path_as_text_id = str(rel_audio_path).replace("/", "_")
    pitch_filepath = os.path.join(pitch_dir, f"{rel_audio_path_as_text_id}.pt")
    pitch = torch.load(pitch_filepath)
    pitch -= pitch_mean
    pitch[pitch == -pitch_mean] = 0.0
    pitch /= pitch_std
    pitch = pitch.unsqueeze(0).to(device=spec_model.device)
    attn_prior = torch.from_numpy(beta_binomial_interpolator(spect_len.item(), text_len.item())).unsqueeze(0).to(spec_model.device)
    
    speaker = torch.tensor([record['speaker']]).to(spec_model.device)
    
    ref_pool = speaker_to_index[record["speaker"]] - set([index]) if len(speaker_to_index[record["speaker"]]) > 1 else speaker_to_index[record["speaker"]]
    ref_sample = manifest[random.sample(ref_pool, 1)[0]]
    gst_ref_audio = wave_model.process(ref_sample["audio_filepath"]).unsqueeze(0).to(device=spec_model.device)
    gst_ref_audio_length = torch.tensor(gst_ref_audio.shape[1]).long().unsqueeze(0).to(device=spec_model.device)
    gst_ref_spec, gst_ref_spec_len = spec_model.preprocessor(input_signal=gst_ref_audio, length=gst_ref_audio_length)  
    
    ref_pool = speaker_to_index[record["speaker"]] - set([index]) if len(speaker_to_index[record["speaker"]]) > 1 else speaker_to_index[record["speaker"]]
    ref_sample_index = random.sample(ref_pool, 1)[0]
    speaker_emb = torch.from_numpy(speaker_embedding[ref_sample_index]).unsqueeze(0).to(spec_model.device)

        
    with torch.no_grad():
        spectrogram = spec_model.forward(
          text=text, 
          input_lens=text_len,
          pitch=pitch,
          spec=spect, 
          mel_lens=spect_len, 
          attn_prior=attn_prior,
          speaker=speaker,
          gst_ref_spec=gst_ref_spec,
          gst_ref_spec_lens=gst_ref_spec_len,
          speaker_embedding=speaker_emb,
        )[0]
    
    spec = spectrogram[0].to('cpu').numpy()
    np.save(save_path, spec)
    return save_path


In [ ]:
spec_model = FastPitchModel.load_from_checkpoint("YOUR_PRETRAINED_FASTPITCH_CHECKPOINT")
spec_model.eval().cuda()
wave_model = WaveformFeaturizer(sample_rate=44100)
beta_binomial_interpolator = BetaBinomialInterpolator()

In [ ]:
os.makedirs(melsdir, exist_ok=True)

# Train
train_datas = json_reader(train_manifest)
base_data_dir = get_base_dir([item["audio_filepath"] for item in train_datas])
train_speaker_embedding = np.load(f'{suppdir}/speaker_embedding/train.npy')

speaker_to_index = defaultdict(list)
for i, d in enumerate(train_datas): speaker_to_index[d.get('speaker', None)].append(i)
speaker_to_index = {k: set(v) for k, v in speaker_to_index.items()}

for i, record in enumerate(tqdm(train_datas)):
    record["mel_filepath"] =  gen_spectrogram(i, train_datas, train_speaker_embedding, speaker_to_index, melsdir, pitch_dir, base_data_dir)

json_writer(train_datas, train_manifest)


# Valid
valid_datas = json_reader(valid_manifest)
base_data_dir = get_base_dir([item["audio_filepath"] for item in valid_datas])
valid_speaker_embedding = np.load(f'{suppdir}/speaker_embedding/valid.npy')

speaker_to_index = defaultdict(list)
for i, d in enumerate(valid_datas): speaker_to_index[d.get('speaker', None)].append(i)
speaker_to_index = {k: set(v) for k, v in speaker_to_index.items()}

for i, record in enumerate(tqdm(valid_datas)):
    record["mel_filepath"] =  gen_spectrogram(i, valid_datas, valid_speaker_embedding, speaker_to_index, melsdir, pitch_dir, base_data_dir)

json_writer(valid_datas, valid_manifest)

## b. Training

In [ ]:
!cd {confdir} \
&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/tts/conf/hifigan/hifigan_44100.yaml
!cd {confdir} \
&& mkdir -p model/train_ds && cd model/train_ds && wget https://raw.githubusercontent.com/nvidia/NeMo/$BRANCH/examples/tts/conf/hifigan/model/train_ds/train_ds_finetune.yaml 
!cd {confdir} \
&& mkdir -p model/validation_ds && cd model/validation_ds && wget https://raw.githubusercontent.com/nvidia/NeMo/$BRANCH/examples/tts/conf/hifigan/model/validation_ds/val_ds_finetune.yaml
!cd {confdir} \
&& mkdir -p model/generator && cd model/generator && wget https://raw.githubusercontent.com/nvidia/NeMo/$BRANCH/examples/tts/conf/hifigan/model/generator/v1_44100.yaml
!cd {codedir} \
&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/tts/hifigan_finetune.py

In [ ]:
# Normally 100 epochs

!(python {codedir}/hifigan_finetune.py \
--config-path={os.path.abspath(confdir)} \
--config-name=hifigan_44100.yaml \
train_dataset={train_manifest} \
validation_datasets={valid_manifest} \
+init_from_pretrained_model="tts_en_hifitts_hifigan_ft_fastpitch" \
model.train_ds.dataloader_params.batch_size=32 \
model.optim.lr=0.0001 \
+trainer.max_epochs=100 \
trainer.check_val_every_n_epoch=20 \
model/train_ds=train_ds_finetune \
model/validation_ds=val_ds_finetune \
trainer.devices=-1 \
trainer.strategy='ddp' \
trainer.precision=16 \
exp_manager.exp_dir={logsdir} \
exp_manager.create_wandb_logger=True \
exp_manager.wandb_logger_kwargs.name="tutorial-HiFiGAN-finetune-multispeaker" \
exp_manager.wandb_logger_kwargs.project="NeMo" \
)

# 3. Fine-tune FastPitch on adaptation data

## a. Data Preparation
For our tutorial, we use a new target speaker from LJSpeech dataset. Usually, the audios should have total duration more than 15 mintues.

In [ ]:
!(cd {codedir} \
&& wget https://raw.githubusercontent.com/nvidia/NeMo/$BRANCH/scripts/dataset_processing/tts/ljspeech/get_data.py -O get_data_ljspeech.py \
&& wget https://raw.githubusercontent.com/nvidia/NeMo/$BRANCH/scripts/dataset_processing/tts/ljspeech/lj_speech.tsv \
)

In [ ]:
!(cd {codedir} && python get_data_ljspeech.py \
        --data-root ../{datadir}/ )

In [ ]:
manidir = f"{datadir}/LJSpeech-1.1"
!ls {manidir}

For simplicity, we use original dev set as training set and original test set as validation set.

In [ ]:
train_manifest = os.path.abspath(os.path.join(manidir, 'val_manifest.json'))
valid_manifest = os.path.abspath(os.path.join(manidir, 'test_manifest.json'))

## b. Preprocessing

### Add absolute file path in manifest

In [ ]:
train_datas = json_reader(train_manifest)
for m in train_datas: m['audio_filepath'] = os.path.abspath(m['audio_filepath'])
json_writer(train_datas, train_manifest)

In [ ]:
valid_datas = json_reader(valid_manifest)
for m in valid_datas: m['audio_filepath'] = os.path.abspath(m['audio_filepath'])
json_writer(valid_datas, valid_manifest)

### Calculate Pitch Stats

In [ ]:
wave_model = WaveformFeaturizer(sample_rate=44100)
pitch_dir = os.path.join(suppdir, 'pitch')
os.makedirs(pitch_dir, exist_ok=True)

train_pitchs = []
train_datas = json_reader(train_manifest)
base_data_dir = get_base_dir([item["audio_filepath"] for item in train_datas])
for m in tqdm(train_datas): train_pitchs.append(get_pitch(m))
    
valid_datas = json_reader(valid_manifest)
base_data_dir = get_base_dir([item["audio_filepath"] for item in valid_datas])
for m in tqdm(valid_datas): get_pitch(m)

train_pitchs = np.concatenate(train_pitchs)
pitch_mean = float(np.mean(train_pitchs))
pitch_std = float(np.std(train_pitchs))

with open(os.path.join(manidir, 'pitch_stats.json'), 'w') as f:
    json.dump({'pitch':[pitch_mean, pitch_std]}, f)

### Extract speaker embedding from pre-trained speaker-verification model 

In [ ]:
train_datas = json_reader(train_manifest)
for m in train_datas: 
    if 'speaker' in m:
        m['old_speaker'] = m['speaker']
    m['speaker'] = 0
    m['label'] = 0
    
json_writer(train_datas, train_manifest)

valid_datas = json_reader(valid_manifest)
for m in valid_datas: 
    if 'speaker' in m:
        m['old_speaker'] = m['speaker']
    m['speaker'] = 0
    m['label'] = 0
    
json_writer(valid_datas, valid_manifest)

In [ ]:
embedding_path = os.path.join(suppdir, 'speaker_embedding_ft')
os.makedirs(embedding_path, exist_ok=True)

# Train
embs, *_ = verification_model.batch_inference(manifest_filepath=train_manifest,
                                              batch_size=32, 
                                              sample_rate=16000, 
                                              device='cuda')
np.save(os.path.join(embedding_path, 'train.npy'), embs)

# Valid
embs, *_ = verification_model.batch_inference(manifest_filepath=valid_manifest,
                                              batch_size=32, 
                                              sample_rate=16000, 
                                              device='cuda')
np.save(os.path.join(embedding_path, 'valid.npy'), embs)

## c. Training

In [ ]:
!cd {codedir} \
&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/tts/fastpitch_finetune_adapters.py

In [ ]:
# Normally 100 epochs

!(python {codedir}/fastpitch_finetune_adapters.py \
--config-path={os.path.abspath(confdir)} \
--config-name=fastpitch_speaker_adaptation.yaml \
sample_rate=44100 \
train_dataset={train_manifest} \
validation_datasets={valid_manifest} \
sup_data_types="['align_prior_matrix', 'pitch', 'speaker_id', 'gst_ref_audio','speaker_embedding']" \
sup_data_path={suppdir} \
+init_from_ptl_ckpt="YOUR_PRETRAINED_FASTPITCH_CHECKPOINT" \
pitch_mean={pitch_mean} \
pitch_std={pitch_std} \
model.n_speakers=118 \
model.speaker_emb_condition_prosody=True \
model.speaker_emb_condition_decoder=True \
model.speaker_emb_condition_aligner=True \
model.speaker_emb_condition_layernm=True \
model.speaker_embedding_dim=192 \
model.adapter.add_weight_speaker=True \
+model.text_tokenizer.add_blank_at=True \
phoneme_dict_path={normdir}/cmudict-0.7b_nv22.10 \
heteronyms_path={normdir}/heteronyms-052722 \
model.train_ds.dataloader_params.batch_size=8 \
model.validation_ds.dataloader_params.batch_size=8 \
model.train_ds.dataloader_params.num_workers=8 \
model.validation_ds.dataloader_params.num_workers=8 \
+model.train_ds.dataset.speaker_embedding_path={suppdir}/speaker_embedding_ft/train.npy \
+model.validation_ds.dataset.speaker_embedding_path={suppdir}/speaker_embedding_ft/valid.npy \
model.optim.lr=2e-4 \
~model.optim.sched \
model.optim.name=adam \
model.optim.weight_decay=0.0 \
trainer.check_val_every_n_epoch=100 \
trainer.max_epochs=100 \
trainer.log_every_n_steps=1 \
trainer.devices=1 \
trainer.strategy=ddp \
trainer.precision=32 \
exp_manager.exp_dir={logsdir} \
+exp_manager.create_wandb_logger=True \
+exp_manager.wandb_logger_kwargs.name="tutorial-FastPitch-finetune-adaptation" \
+exp_manager.wandb_logger_kwargs.project="NeMo" \
+exp_manager.checkpoint_callback_params.save_top_k=-1 \
)

# 4. Fine-tune HiFiGAN on adaptation data

## a. Dataset Preparation

In [ ]:
# Pretrained FastPitch Weights
spec_model = FastPitchModel.load_from_checkpoint("YOUR_PRETRAINED_FASTPITCH_CHECKPOINT", strict=False)

YOUR_FINETUNED_FASTPITCH_CHECKPOINT = "YOUR_FINETUNED_FASTPITCH_CHECKPOINT"

# Load Adapter Weights
spec_model.load_adapters(f'{logsdir}/FastPitch/{YOUR_FINETUNED_FASTPITCH_CHECKPOINT}/checkpoints/adapters.pt')
spec_model.freeze()
spec_model.unfreeze_enabled_adapters()

# Load Weighted Speaker Embedding Weights
wemb = WeightedSpeakerEmbedding(pretrained_embedding=spec_model.fastpitch.speaker_emb)
wemb.embedding_weight.data = torch.load(f'{logsdir}/FastPitch/{YOUR_FINETUNED_FASTPITCH_CHECKPOINT}/checkpoints/wemb.pt')
wemb.embedding_weight.requires_grad = False
spec_model.fastpitch.speaker_emb = wemb
spec_model.eval().cuda()

In [ ]:
wave_model = WaveformFeaturizer(sample_rate=44100)
beta_binomial_interpolator = BetaBinomialInterpolator()

In [ ]:
os.makedirs(melsdir, exist_ok=True)

# Train
train_datas = json_reader(train_manifest)
base_data_dir = get_base_dir([item["audio_filepath"] for item in train_datas])
train_speaker_embedding = np.load(f'{suppdir}/speaker_embedding_ft/train.npy')
speaker_to_index = defaultdict(list)
for i, d in enumerate(train_datas): speaker_to_index[d.get('speaker', None)].append(i)
speaker_to_index = {k: set(v) for k, v in speaker_to_index.items()}

for i, record in enumerate(tqdm(train_datas)):
    record["mel_filepath"] =  gen_spectrogram(i, train_datas, train_speaker_embedding, speaker_to_index, melsdir, pitch_dir, base_data_dir)

json_writer(train_datas, train_manifest)


# Valid
valid_datas = json_reader(valid_manifest)
base_data_dir = get_base_dir([item["audio_filepath"] for item in valid_datas])
valid_speaker_embedding = np.load(f'{suppdir}/speaker_embedding_ft/valid.npy')
speaker_to_index = defaultdict(list)
for i, d in enumerate(valid_datas): speaker_to_index[d.get('speaker', None)].append(i)
speaker_to_index = {k: set(v) for k, v in speaker_to_index.items()}

for i, record in enumerate(tqdm(valid_datas)):
    record["mel_filepath"] =  gen_spectrogram(i, valid_datas, valid_speaker_embedding, speaker_to_index, melsdir, pitch_dir, base_data_dir)

json_writer(valid_datas, valid_manifest)

## b. Training

In [ ]:
# Normally 500 epochs 
!(python {codedir}/hifigan_finetune.py \
--config-path={os.path.abspath(confdir)} \
--config-name=hifigan_44100.yaml \
train_dataset={train_manifest} \
validation_datasets={valid_manifest} \
+init_from_ptl_ckpt="YOUR_FINETUNED_HIFIGAN_ON_MULTISPEAKER_CHECKPOINT" \
model.train_ds.dataloader_params.batch_size=32 \
model.optim.lr=0.0001 \
+trainer.max_epochs=500 \
trainer.check_val_every_n_epoch=20 \
model/train_ds=train_ds_finetune \
model/validation_ds=val_ds_finetune \
trainer.devices=1 \
trainer.strategy='ddp' \
trainer.precision=16 \
exp_manager.exp_dir={logsdir} \
exp_manager.create_wandb_logger=True \
exp_manager.wandb_logger_kwargs.name="tutorial-HiFiGAN" \
exp_manager.wandb_logger_kwargs.project="NeMo" \
)

# 5. Inference

In [ ]:
from nemo.collections.tts.models import HifiGanModel
import IPython.display as ipd

## a. Load Model

In [ ]:
# Waveform
sample_rate = 44100
wave_model = WaveformFeaturizer(sample_rate=sample_rate)

# FastPitch
spec_model = FastPitchModel.load_from_checkpoint(last_ckpt, strict=False)
spec_model.load_adapters(f'{logsdir}/FastPitch/{YOUR_FINETUNED_FASTPITCH_CHECKPOINT}/checkpoints/adapters.pt')
spec_model.freeze()
spec_model.unfreeze_enabled_adapters()
wemb = WeightedSpeakerEmbedding(pretrained_embedding=spec_model.fastpitch.speaker_emb)
wemb.embedding_weight.data = torch.load(f'{logsdir}/FastPitch/{YOUR_FINETUNED_FASTPITCH_CHECKPOINT}/checkpoints/wemb.pt')
wemb.embedding_weight.requires_grad = False
spec_model.fastpitch.speaker_emb = wemb
spec_model.eval().cuda()

# HiFiGAN
vocoder_model = HifiGanModel.load_from_checkpoint(checkpoint_path=f'YOUR_FINETUNED_HIFIGAN_ON_ADAPTATION_CHECKPOINT')
vocoder_model.eval().cuda()

## b. Output Audio

In [ ]:
def gt_spectrogram(audio_path, wave_model, spec_gen_model):
    features = wave_model.process(audio_path, trim=False)
    audio, audio_length = features, torch.tensor(features.shape[0]).long()
    audio = audio.unsqueeze(0).to(device=spec_gen_model.device)
    audio_length = audio_length.unsqueeze(0).to(device=spec_gen_model.device)
    with torch.no_grad():
        spectrogram, spec_len = spec_gen_model.preprocessor(input_signal=audio, length=audio_length)
    return spectrogram, spec_len

def gen_spectrogram(text, spec_gen_model, speaker, spec_ref, spec_ref_lens, speaker_embedding):
    parsed = spec_gen_model.parse(text)
    speaker = torch.tensor([speaker]).long().to(device=spec_gen_model.device)
    with torch.no_grad():    
        spectrogram = spec_gen_model.generate_spectrogram(tokens=parsed, 
                                                          speaker=speaker, 
                                                          speaker_embedding=speaker_embedding,
                                                          gst_ref_spec=spec_ref, 
                                                          gst_ref_spec_lens=spec_ref_lens)

    return spectrogram
  
def sync_audio(vocoder_model, spectrogram):    
    with torch.no_grad():  
        audio = vocoder_model.convert_spectrogram_to_audio(spec=spectrogram)
    if isinstance(audio, torch.Tensor):
        audio = audio.to('cpu').numpy()
    return audio

In [ ]:
os.makedirs(gensdir, exist_ok=True)

# Reference Audio
with open(train_manifest, "r") as f:
    for i, line in enumerate(f):
        ref_record = json.loads(line)
        break
        
# Validatation Audio
num_val = 10
val_records = []
with open(valid_manifest, "r") as f:
    for i, line in enumerate(f):
        val_records.append(json.loads(line))
        if len(val_records) >= num_val:
            break
            
# Speaker Embedding
speaker_embeddings = np.load(f'{suppdir}/speaker_embedding_ft/train.npy')
speaker_embedding  = np.mean(speaker_embeddings, axis=0)
speaker_embedding  = torch.from_numpy(speaker_embedding).unsqueeze(0).to(spec_model.device)

In [ ]:
for i, val_record in enumerate(val_records):
    with open(os.path.join(gensdir, f'{i}-text.txt'), 'w') as f: f.write(val_record['text'])

    audio = ipd.Audio(val_record['audio_filepath'], rate=sample_rate)
    with open(os.path.join(gensdir, f'{i}-GT.wav'), 'wb') as f: f.write(audio.data)

    spec_ref, spec_ref_len = gt_spectrogram(ref_record['audio_filepath'], wave_model, spec_model)
    spec_ref = spec_ref.to(spec_model.device)
    spec_pred = gen_spectrogram(val_record['text'], spec_model,
                                speaker=val_record['speaker'], 
                                speaker_embedding=speaker_embedding,
                                spec_ref=spec_ref, spec_ref_lens=spec_ref_len)
    
    audio = sync_audio(vocoder_model, spec_pred)
    audio = ipd.Audio(audio, rate=sample_rate)
    with open(os.path.join(gensdir, f'{i}-Gen.wav'), 'wb') as f: f.write(audio.data)